<a href="https://colab.research.google.com/github/casonshep/ztfquery-algorithm-shepard/blob/main/Finished_Algorithm(fast).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8
from IPython.display import clear_output 
clear_output()
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

# Choose one of the given alternatives:
!sudo update-alternatives --config python3

# This one used to work but now NOT(for me)!
# !sudo update-alternatives --config python

# Check the result
!python3 --version



In [ ]:
# setup, run on startup/reconnect
!python -m pip install ztfquery
!python -m pip install astrobject


%matplotlib notebook

In [ ]:
from ztfquery import query, buildurl, io
from astropy import time, units as u
from numpy import random, average
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
import csv, time as time2
import pandas as sorting

from google.colab import output

# to get the last nights date
from datetime import date, timedelta

In [ ]:
header= ["grid", "objid", "ra", "dec"]

# turns csv to array of lists called data_2d
with open('/content/data/data_sort') as file:
  data_2d = []
  reader = csv.reader(file, delimiter=',')
  next(reader) # this will skip the header
  for row in reader:
    data_2d.append(row) 
data_2d = data_2d[0:len(data_2d)-1]

# generates csv file for each gridid in the data file
# also creates a list of fields for use with ztf query later
field_list = []
for x in data_2d:
  field_list.append(int(float(x[0])))
  temp = data_2d[data_2d.index(x)-1];
  with open('/content/data/grid_'+str(int(float(x[0]))), 'a') as grid_file:
    writer = csv.writer(grid_file)
    if int(float(temp[0])) != int(float(x[0])):
      writer.writerow(header)
    writer.writerow(x)
field_list = [*set(field_list)]
field_list.sort
output.clear()

In [ ]:
day = date.today() - timedelta(days=1)
day_str = day.strftime("%Y-%m-%d")

zquery = query.ZTFQuery()

jdate = time.Time(day_str).jd

start = time2.time()

field_update_count = 0
object_update_count = 0

for x in field_list:

  if field_list.index(x) != 0:
    print(f'Completed query of grid {field_list[field_list.index(x)-1]}, this is field {field_list.index(x)}/{len(field_list)}')
  
  zquery.load_metadata(kind="sci", sql_query=f'obsjd >= {jdate} and field = {x}')
  
  if len(zquery.metatable['ra']) != 0:
    #only updates the update count if the metadata query is nonempty
    field_update_count = field_update_count + 1

    float_ra = [float(i) for i in list(zquery.metatable['ra'])]
    float_dec = [float(f) for f in list(zquery.metatable['dec'])]
    metadata_catalog = SkyCoord(float_ra*u.degree, float_dec*u.degree)

    zquery.get_data_path(suffix="psfcat.fits")
    urls, locations = zquery.download_data(suffix="psfcat.fits", source="local", nprocess=8,
                                      download_dir='/content/ztf_out/', show_progress=True, nodl=True,)
  
    #creates field_data list which can be used as original skycoord to find measurement match
    with open('/content/data/grid_'+str(x)) as file:
      field_data = []
      reader = csv.reader(file, delimiter=',')
      next(reader) # this will skip the header
      for row in reader:
        field_data.append(row)
      
      object_update_count = object_update_count + len(field_data)

      #create csv to store data
      with open('/content/data_out/csv_'+str(x), 'w') as file:
        writer = csv.writer(file)

        header = ['objid', 'ra', 'dec', 'flux', 'sigflux', 'mag', 'sigmag']
        writer.writerow(header)     
        for obj in field_data:

          #creates reference SkyCoord object
          original = SkyCoord(ra=float(obj[2])*u.degree, dec=float(obj[3])*u.degree)
          index_meta_match, angle, quant = match_coordinates_sky(original, metadata_catalog, nthneighbor=1)

          #load metadata and download for given cooridnate query
          temp_url = buildurl.filename_to_scienceurl(locations[index_meta_match])
          io.download_single_url(temp_url,locations[index_meta_match])
          with fits.open(locations[index_meta_match]) as hdu:
            data = hdu[1].data
          
            # creates catalog of coordinate pairs
            float_ra = [float(i) for i in data['ra']]
            float_dec = [float(f) for f in data['dec']]
            catalog = SkyCoord(float_ra*u.degree, float_dec*u.degree)

            # calculates the closest pair and stores coordinate in data_match
            index_match, angle, quant = match_coordinates_sky(original, catalog, nthneighbor=1)
            data_match = data[index_match]
            # writes coordinate matched data to csv file
            newline = [obj[1], data_match['ra'], data_match['dec'], data_match['flux'],
                     data_match['sigflux'], data_match['mag'], data_match['sigmag']]
            writer.writerow(newline)

end = time2.time()
print(f'{len(field_list)} queries completed in {end-start}.') 
print(f'Transients from {field_update_count} sky grids were updated in the past 24 hours.')  
print(f'Data from a total of {object_update_count} transient objects was updated.')       